<a href="https://colab.research.google.com/github/Osw1997/An-Introduction-to-Statistical-Learning/blob/main/Scripts/Classification/applied_exercises_ch4_classification.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Chapter 4 - Classification


# Applied exercises
---
---
---
## Exercise 13
## Involved dataset: Weekly
13. This question should be answered using the Weekly data set, which is part of the ISLR2 package. This data is similar in nature to the Smarket data from this chapter’s lab, except that it contains 1, 089 weekly returns for 21 years, from the beginning of 1990 to the end of 2010.

In [1]:
!pip install --upgrade scipy
!pip install --upgrade plotly

     |████████████████████████████████| 38.1 MB 1.6 MB/s 
  Attempting uninstall: scipy
    Found existing installation: scipy 1.4.1
    Uninstalling scipy-1.4.1:
      Successfully uninstalled scipy-1.4.1
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
albumentations 0.1.12 requires imgaug<0.2.7,>=0.2.5, but you have imgaug 0.2.9 which is incompatible.
     |████████████████████████████████| 27.7 MB 15.0 MB/s 
  Attempting uninstall: plotly
    Found existing installation: plotly 5.5.0
    Uninstalling plotly-5.5.0:
      Successfully uninstalled plotly-5.5.0


In [2]:
import pandas as pd
import os
import matplotlib.pyplot as plt
from scipy import stats
from scipy.stats import t

In [3]:
import scipy
scipy.__version__

'1.7.3'

In [4]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [5]:
weekly = pd.read_csv('drive/MyDrive/itsl_datasets/Weekly.csv')
weekly.columns

Index(['Year', 'Lag1', 'Lag2', 'Lag3', 'Lag4', 'Lag5', 'Volume', 'Today',
       'Direction'],
      dtype='object')

In [6]:
weekly.describe()

,Year,Lag1,Lag2,Lag3,Lag4,Lag5,Volume,Today
count,1089.000000,1089.000000,1089.000000,1089.000000,1089.000000,1089.000000,1089.000000,1089.000000
mean,2000.048669,0.150585,0.151079,0.147205,0.145818,0.139893,1.574618,0.149899
std,6.033182,2.357013,2.357254,2.360502,2.360279,2.361285,1.686636,2.356927
min,1990.000000,-18.195000,-18.195000,-18.195000,-18.195000,-18.195000,0.087465,-18.195000
25%,1995.000000,-1.154000,-1.154000,-1.158000,-1.158000,-1.166000,0.332022,-1.154000
50%,2000.000000,0.241000,0.241000,0.241000,0.238000,0.234000,1.002680,0.241000
75%,2005.000000,1.405000,1.409000,1.409000,1.409000,1.405000,2.053727,1.405000
max,2010.000000,12.026000,12.026000,12.026000,12.026000,12.026000,9.328214,12.026000


In [7]:
weekly.head()

,Year,Lag1,Lag2,Lag3,Lag4,Lag5,Volume,Today,Direction
0,1990,0.816,1.572,-3.936,-0.229,-3.484,0.154976,-0.270,Down
1,1990,-0.270,0.816,1.572,-3.936,-0.229,0.148574,-2.576,Down
2,1990,-2.576,-0.270,0.816,1.572,-3.936,0.159837,3.514,Up
3,1990,3.514,-2.576,-0.270,0.816,1.572,0.161630,0.712,Up
4,1990,0.712,3.514,-2.576,-0.270,0.816,0.153728,1.178,Up


In [8]:
weekly['Direction'].unique() # Down, Up
# One one encoding (ohe)
weekly_ohe = weekly.copy()
weekly_ohe['Direction'] = pd.get_dummies(weekly_ohe['Direction']).drop('Down', axis=1)
weekly_ohe.describe()

,Year,Lag1,Lag2,Lag3,Lag4,Lag5,Volume,Today,Direction
count,1089.000000,1089.000000,1089.000000,1089.000000,1089.000000,1089.000000,1089.000000,1089.000000,1089.000000
mean,2000.048669,0.150585,0.151079,0.147205,0.145818,0.139893,1.574618,0.149899,0.555556
std,6.033182,2.357013,2.357254,2.360502,2.360279,2.361285,1.686636,2.356927,0.497132
min,1990.000000,-18.195000,-18.195000,-18.195000,-18.195000,-18.195000,0.087465,-18.195000,0.000000
25%,1995.000000,-1.154000,-1.154000,-1.158000,-1.158000,-1.166000,0.332022,-1.154000,0.000000
50%,2000.000000,0.241000,0.241000,0.241000,0.238000,0.234000,1.002680,0.241000,1.000000
75%,2005.000000,1.405000,1.409000,1.409000,1.409000,1.405000,2.053727,1.405000,1.000000
max,2010.000000,12.026000,12.026000,12.026000,12.026000,12.026000,9.328214,12.026000,1.000000


In [9]:
weekly_ohe.head()

,Year,Lag1,Lag2,Lag3,Lag4,Lag5,Volume,Today,Direction
0,1990,0.816,1.572,-3.936,-0.229,-3.484,0.154976,-0.270,0
1,1990,-0.270,0.816,1.572,-3.936,-0.229,0.148574,-2.576,0
2,1990,-2.576,-0.270,0.816,1.572,-3.936,0.159837,3.514,1
3,1990,3.514,-2.576,-0.270,0.816,1.572,0.161630,0.712,1
4,1990,0.712,3.514,-2.576,-0.270,0.816,0.153728,1.178,1


In [10]:

# (a) Produce some numerical and graphical summaries of the Weekly data. Do there appear to be any patterns?
import plotly.express as px
numeric_columns = ['Year', 'Lag1', 'Lag2', 'Lag3', 'Lag4', 'Lag5', 'Volume'
                   , 'Today'
                   , 'Direction'
                   ]

fig = px.scatter_matrix(weekly_ohe[numeric_columns])
fig.update_traces(diagonal_visible=False)
fig.show()



import plotly.express as px
import numpy as np
corr_matrix = np.corrcoef(weekly_ohe[numeric_columns], rowvar=False)

# corr_matrix

fig = px.imshow(corr_matrix
                , text_auto=True
                , labels=dict(x="Columns", y="Columns")
                , x=numeric_columns
                , y=numeric_columns
                , aspect="auto"
                , color_continuous_scale="BuPu"
                )
fig.update_xaxes(side="top")
fig.show()
# It seems that exists a correlation between the column "Volume" and "Year", 
# and "Direction" and "Today".


In [11]:
# (b) Use the full data set to perform a logistic regression with Direction as the response and the five lag variables
#  plus Volume as predictors. Use the summary function to print the results. Do any of the predictors appear to be statistically significant? 
#  If so, which ones?
# from sklearn.linear_model import LogisticRegression
# lr = LogisticRegression()
# response = ['Direction']
# predictors = ['Lag1', 'Lag2', 'Lag3', 'Lag4', 'Lag5', 'Volume']
# lr.fit(X=weekly_ohe[predictors], y=weekly[response])

import statsmodels.formula.api as smf
log_reg = smf.logit(formula="Direction ~ Lag1 + Lag2 + Lag3 + Lag4 + Lag5 + Volume", data=weekly_ohe).fit()
log_reg.summary2()
# # Predictors statistically significant:
# Intersect & Lag2

# (c) Compute the confusion matrix and overall fraction of correct predictions. 
# Explain what the confusion matrix is telling you about the types of mistakes made by logistic regression.
from sklearn.metrics import confusion_matrix
threshold = 0.5
y_pred = log_reg.predict() > threshold
conf_mat = confusion_matrix(weekly_ohe['Direction'], y_pred)
print(conf_mat)
print(f"Overall fraction of correct predictions: {conf_mat[0][0]} + {conf_mat[1][1]} / {len(y_pred)} =  {(conf_mat[0][0] + conf_mat[1][1])/len(y_pred)}")
# We can see that our predictor is better that guessing the outcome. (0.56 > 0.5)


/usr/local/lib/python3.7/dist-packages/statsmodels/tools/_testing.py:19: FutureWarning:

pandas.util.testing is deprecated. Use the functions in the public API at pandas.testing instead.



Optimization terminated successfully.
         Current function value: 0.682441
         Iterations 4
[[ 54 430]
 [ 48 557]]
Overall fraction of correct predictions: 54 + 557 / 1089 =  0.5610651974288338


In [12]:
# (d) Now fit the logistic regression model using a training data period from 1990 to 2008, with Lag2 as the only predictor.
weekly_ohe_90_08 = weekly_ohe[(weekly_ohe['Year'] >= 1990) & (weekly_ohe['Year'] <= 2008)]
log_reg_v2 = smf.logit(formula="Direction ~ Lag2", data=weekly_ohe_90_08).fit()
log_reg_v2.summary2()
#  Compute the confusion matrix and the overall fraction of correct predictions for the held out data (that is, the data from 2009 and 2010).
weekly_ohe_08 = weekly_ohe[weekly_ohe['Year'] > 2008]
threshold = 0.5
y_pred = log_reg_v2.predict(weekly_ohe_08) > threshold
conf_mat = confusion_matrix(weekly_ohe_08['Direction'], y_pred)
print(conf_mat)
print(f"Overall fraction of correct predictions: {conf_mat[0][0]} + {conf_mat[1][1]} / {len(y_pred)} =  {(conf_mat[0][0] + conf_mat[1][1])/len(y_pred)}")


from sklearn.linear_model import LogisticRegression
lr = LogisticRegression()
response = ['Direction']
predictors = ['Lag2']
lr.fit(X=weekly_ohe_90_08[predictors], y=weekly_ohe_90_08[response].values.ravel())
y_pr = lr.predict(X=weekly_ohe_08[predictors])
conf_mat = confusion_matrix(weekly_ohe_08['Direction'], y_pr)
print(conf_mat)
print(f"Overall fraction of correct predictions: {conf_mat[0][0]} + {conf_mat[1][1]} / {len(y_pred)} =  {(conf_mat[0][0] + conf_mat[1][1])/len(y_pred)}")
print(f"Score: {lr.score(X=weekly_ohe_08[predictors], y=weekly_ohe_08[response])}")



Optimization terminated successfully.
         Current function value: 0.685555
         Iterations 4
[[ 9 34]
 [ 5 56]]
Overall fraction of correct predictions: 9 + 56 / 104 =  0.625
[[ 9 34]
 [ 5 56]]
Overall fraction of correct predictions: 9 + 56 / 104 =  0.625
Score: 0.625


In [13]:
weekly_ohe_90_08['Lag2'].shape

(985,)

In [14]:
# (e) Repeat (d) using LDA. LINEAR DISCRIMINANT ANALYSIS
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
print("=" * 30 + ' >>> Linear Discriminant Analysis (LDA) <<<' + "=" * 30)

lda = LinearDiscriminantAnalysis()
response = ['Direction']
predictors = ['Lag2']
lda_trained = lda.fit(X=weekly_ohe_90_08[predictors], y=weekly_ohe_90_08[response].values.ravel())
y_pred = lda_trained.predict(X=weekly_ohe_08[predictors])
conf_matrix = confusion_matrix(weekly_ohe_08[response], y_pred)
print(f"Overall fraction of correct predictions: {conf_mat[0][0]} + {conf_mat[1][1]} / {len(y_pred)} =  {(conf_mat[0][0] + conf_mat[1][1])/len(y_pred)}")
print(f"Score: {lr.score(X=weekly_ohe_08[predictors], y=weekly_ohe_08[response])}")




# (f) Repeat (d) using QDA. QUADRATIC DISCRIMINANT ANALYSIS
from sklearn.discriminant_analysis import QuadraticDiscriminantAnalysis
print("=" * 30 + ' >>> Linear Discriminant Analysis (LDA) <<<' + "=" * 30)

model = QuadraticDiscriminantAnalysis()
response = ['Direction']
predictors = ['Lag2']
model_trained = model.fit(X=weekly_ohe_90_08[predictors], y=weekly_ohe_90_08[response].values.ravel())
y_pred = model_trained.predict(X=weekly_ohe_08[predictors])
conf_matrix = confusion_matrix(weekly_ohe_08[response], y_pred)
print(f"Overall fraction of correct predictions: {conf_mat[0][0]} + {conf_mat[1][1]} / {len(y_pred)} =  {(conf_mat[0][0] + conf_mat[1][1])/len(y_pred)}")
print(f"Score: {lr.score(X=weekly_ohe_08[predictors], y=weekly_ohe_08[response])}")

# (g) Repeat (d) using KNN with K = 1.
from sklearn.neighbors import KNeighborsClassifier
print("=" * 30 + ' >>> K-Neighbors (N=1) <<<' + "=" * 30)

model = KNeighborsClassifier(n_neighbors=1)
response = ['Direction']
predictors = ['Lag2']
model_trained = model.fit(X=weekly_ohe_90_08[predictors], y=weekly_ohe_90_08[response].values.ravel())
y_pred = model_trained.predict(X=weekly_ohe_08[predictors])
conf_matrix = confusion_matrix(weekly_ohe_08[response], y_pred)
print(f"Overall fraction of correct predictions: {conf_mat[0][0]} + {conf_mat[1][1]} / {len(y_pred)} =  {(conf_mat[0][0] + conf_mat[1][1])/len(y_pred)}")
print(f"Score: {lr.score(X=weekly_ohe_08[predictors], y=weekly_ohe_08[response])}")

# (h) Repeat (d) using naive Bayes.
from sklearn.naive_bayes import GaussianNB
print("=" * 30 + ' >>> Naive Bayes <<<' + "=" * 30)

model = GaussianNB()
response = ['Direction']
predictors = ['Lag2']
model_trained = model.fit(X=weekly_ohe_90_08[predictors], y=weekly_ohe_90_08[response].values.ravel())
y_pred = model_trained.predict(X=weekly_ohe_08[predictors])
conf_matrix = confusion_matrix(weekly_ohe_08[response], y_pred)
print(f"Overall fraction of correct predictions: {conf_mat[0][0]} + {conf_mat[1][1]} / {len(y_pred)} =  {(conf_mat[0][0] + conf_mat[1][1])/len(y_pred)}")
print(f"Score: {lr.score(X=weekly_ohe_08[predictors], y=weekly_ohe_08[response])}")

============================== >>> Linear Discriminant Analysis (LDA) <<<==============================
Overall fraction of correct predictions: 9 + 56 / 104 =  0.625
Score: 0.625
============================== >>> Linear Discriminant Analysis (LDA) <<<==============================
Overall fraction of correct predictions: 9 + 56 / 104 =  0.625
Score: 0.625
============================== >>> K-Neighbors (N=1) <<<==============================
Overall fraction of correct predictions: 9 + 56 / 104 =  0.625
Score: 0.625
============================== >>> Naive Bayes <<<==============================
Overall fraction of correct predictions: 9 + 56 / 104 =  0.625
Score: 0.625


## (i) Which of these methods appears to provide the best results on this data?
### **A = All of them has the same score (0.625) but it's better to chose the method that implies less computational cost.**

### (j) Experiment with different combinations of predictors, including possible transformations and interactions, for each of the methods. Report the variables, method, and associated confusion matrix that appears to provide the best results on the held out data. Note that you should also experiment with values for K in the KNN classifier.

---
---
---
## Exercise 14
## Involved dataset: Auto
14. In this problem, you will develop a model to predict whether a given
car gets high or low gas mileage based on the Auto data set.


In [15]:
auto = pd.read_csv('drive/MyDrive/itsl_datasets/Auto.csv')
# auto = pd.read_csv("../../Datasets/Auto.csv")
## Cleaning data
# Getting incorrect rows and getting mean of correct rows
valid_rows = auto['horsepower'] != '?'
mean_valid_rows = pd.to_numeric(auto[valid_rows]['horsepower']).mean()
# Correcting data
auto[~valid_rows] = mean_valid_rows
auto['horsepower'] = pd.to_numeric(auto['horsepower'])
auto['mpg'] = pd.to_numeric(auto['mpg'])

auto.describe()
# auto.head()

,mpg,cylinders,displacement,horsepower,weight,acceleration,year,origin
count,397.000000,397.000000,397.000000,397.000000,397.000000,397.000000,397.000000,397.000000
mean,24.466365,6.718758,193.279211,104.469388,2941.398859,16.661327,76.338405,2.872410
std,11.916122,11.182938,104.465120,38.247388,902.933828,10.300906,4.849520,11.516536
min,9.000000,3.000000,68.000000,46.000000,104.469388,8.000000,70.000000,1.000000
25%,17.500000,4.000000,104.469388,76.000000,2219.000000,13.800000,73.000000,1.000000
50%,23.000000,4.000000,146.000000,95.000000,2790.000000,15.500000,76.000000,1.000000
75%,29.500000,8.000000,262.000000,125.000000,3609.000000,17.300000,79.000000,2.000000
max,104.469388,104.469388,455.000000,230.000000,5140.000000,104.469388,104.469388,104.469388


In [16]:
auto.head()

,mpg,cylinders,displacement,horsepower,weight,acceleration,year,origin,name
0,18.0,8.0,307.0,130.0,3504.0,12.0,70.0,1.0,chevrolet chevelle malibu
1,15.0,8.0,350.0,165.0,3693.0,11.5,70.0,1.0,buick skylark 320
2,18.0,8.0,318.0,150.0,3436.0,11.0,70.0,1.0,plymouth satellite
3,16.0,8.0,304.0,150.0,3433.0,12.0,70.0,1.0,amc rebel sst
4,17.0,8.0,302.0,140.0,3449.0,10.5,70.0,1.0,ford torino


In [17]:
# (a) Create a binary variable, mpg01, that contains a 1 if mpg contains a value above its median, and a 0 if mpg contains a value below
# its median. You can compute the median using the median() function. Note you may find it helpful to use the data.frame()
# function to create a single data set containing both mpg01 and the other Auto variables.
median_mpg = np.median(auto['mpg'])
auto['mpg0'] = (auto['mpg'] > median_mpg).astype(int)

# (b) Explore the data graphically in order to investigate the association between mpg01 and the other features. Which of the other
# features seem most likely to be useful in predicting mpg01? Scatterplots and boxplots may be useful tools to answer this question. 
# Describe your findings.
import plotly.express as px
numeric_columns = ['mpg0', 'cylinders', 'displacement', 'horsepower', 
                   'weight', 'acceleration', 'year', 'origin']

fig = px.scatter_matrix(auto[numeric_columns])
fig.update_traces(diagonal_visible=False)
fig.show()



import plotly.express as px
import numpy as np
corr_matrix = np.corrcoef(auto[numeric_columns], rowvar=False)

# corr_matrix
fig = px.imshow(corr_matrix
                , text_auto=True
                , labels=dict(x="Columns", y="Columns")
                , x=numeric_columns
                , y=numeric_columns
                , aspect="auto"
                , color_continuous_scale="BuPu"
                )
fig.update_xaxes(side="top")
fig.show()

### Correlations
## Positive
# year            (+0.41)
## Negative
# displacement    (-0.74)
# weight          (-0.64)
# horsepower      (-0.73)


In [18]:
# (c) Split the data into a training set and a test set.
from sklearn.model_selection import train_test_split
predictors = ['cylinders', 'displacement', 'horsepower', 
              'weight', 'acceleration', 'year', 'origin']
response = ['mpg0']
X_train, X_test, y_train, y_test = train_test_split(auto[predictors], auto[response],
                                                    test_size=0.25, random_state=10)


In [19]:
# (d) Perform LDA on the training data in order to predict mpg01 using the variables that seemed most associated with mpg01 in
# (b). What is the test error of the model obtained?
### Correlations
## Positive
# year            (+0.41)
## Negative
# displacement    (-0.74)
# weight          (-0.64)
# horsepower      (-0.73)
significant_features = ['year', 'displacement', 'weight', 'horsepower']
response = ['mpg0']

from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
from sklearn.metrics import confusion_matrix

lda = LinearDiscriminantAnalysis()
lda_trained = lda.fit(X=X_train[significant_features], y=y_train.values.ravel())
y_pred = lda_trained.predict(X=X_test[significant_features])

conf_matrix = confusion_matrix(y_test, y_pred)
print(conf_matrix)
print(f"Overall fraction of INCORRECT predictions: {conf_matrix[1][0]} + {conf_matrix[0][1]} / {len(y_pred)} =  {(conf_matrix[1][0] + conf_matrix[0][1])/len(y_pred)}")
print(f"Error: {1 - lda_trained.score(X=X_test[significant_features], y=y_test)}")



[[45 14]
 [ 0 41]]
Overall fraction of INCORRECT predictions: 0 + 14 / 100 =  0.14
Error: 0.14


In [20]:
# (e) Perform QDA on the training data in order to predict mpg01 using the variables that seemed most associated with mpg01 in
# (b). What is the test error of the model obtained?
from sklearn.discriminant_analysis import QuadraticDiscriminantAnalysis
from sklearn.metrics import confusion_matrix

model = QuadraticDiscriminantAnalysis()
model_trained = model.fit(X=X_train[significant_features], y=y_train.values.ravel())
y_pred = model_trained.predict(X=X_test[significant_features])

conf_matrix = confusion_matrix(y_test, y_pred)
print(conf_matrix)
print(f"Overall fraction of INCORRECT predictions: {conf_matrix[1][0]} + {conf_matrix[0][1]} / {len(y_pred)} =  {(conf_matrix[1][0] + conf_matrix[0][1])/len(y_pred)}")
print(f"Error: {1 - model_trained.score(X=X_test[significant_features], y=y_test)}")


[[48 11]
 [ 1 40]]
Overall fraction of INCORRECT predictions: 1 + 11 / 100 =  0.12
Error: 0.12


In [21]:
# (f) Perform logistic regression on the training data in order to predict mpg01 using the variables that seemed most associated with
# mpg01 in (b). What is the test error of the model obtained?
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import confusion_matrix

model = LogisticRegression()
model_trained = model.fit(X=X_train[significant_features], y=y_train.values.ravel())
y_pred = model_trained.predict(X=X_test[significant_features])

conf_matrix = confusion_matrix(y_test, y_pred)
print(conf_matrix)
print(f"Overall fraction of INCORRECT predictions: {conf_matrix[1][0]} + {conf_matrix[0][1]} / {len(y_pred)} =  {(conf_matrix[1][0] + conf_matrix[0][1])/len(y_pred)}")
print(f"Error: {1 - model_trained.score(X=X_test[significant_features], y=y_test)}")


[[52  7]
 [ 0 41]]
Overall fraction of INCORRECT predictions: 0 + 7 / 100 =  0.07
Error: 0.06999999999999995


In [22]:
# (g) Perform naive Bayes on the training data in order to predict mpg01 using the variables that seemed most associated with mpg01
# in (b). What is the test error of the model obtained?
from sklearn.naive_bayes import GaussianNB
from sklearn.metrics import confusion_matrix

model = GaussianNB()
model_trained = model.fit(X=X_train[significant_features], y=y_train.values.ravel())
y_pred = model_trained.predict(X=X_test[significant_features])

conf_matrix = confusion_matrix(y_test, y_pred)
print(conf_matrix)
print(f"Overall fraction of INCORRECT predictions: {conf_matrix[1][0]} + {conf_matrix[0][1]} / {len(y_pred)} =  {(conf_matrix[1][0] + conf_matrix[0][1])/len(y_pred)}")
print(f"Error: {1 - model_trained.score(X=X_test[significant_features], y=y_test)}")


[[47 12]
 [ 0 41]]
Overall fraction of INCORRECT predictions: 0 + 12 / 100 =  0.12
Error: 0.12


In [23]:
# (h) Perform KNN on the training data, with several values of K, in order to predict mpg01. Use only the variables that seemed most
# associated with mpg01 in (b). What test errors do you obtain?
# Which value of K seems to perform the best on this data set?
from sklearn.neighbors import KNeighborsClassifier

prev_min_value = 1000000
for n in range(1, 200):
    # print(f"K-Neighbors : n_neighbors = {n}")
    model = KNeighborsClassifier(n_neighbors=n)
    model_trained = model.fit(X=X_train[significant_features], y=y_train.values.ravel())
    y_pred = model_trained.predict(X=X_test[significant_features])

    conf_matrix = confusion_matrix(y_test, y_pred)
    # print(conf_matrix)
    # print(f"Overall fraction of INCORRECT predictions: {conf_matrix[1][0]} + {conf_matrix[0][1]} / {len(y_pred)} =  {(conf_matrix[1][0] + conf_matrix[0][1])/len(y_pred)}")
    if (1 - model_trained.score(X=X_test[significant_features], y=y_test)) < prev_min_value:
        prev_min_value = 1 - model_trained.score(X=X_test[significant_features], y=y_test)
        print(f"K-Neighbors : n_neighbors = {n}")
        print(f"Error: {prev_min_value}")


K-Neighbors : n_neighbors = 1
Error: 0.10999999999999999
K-Neighbors : n_neighbors = 20
Error: 0.09999999999999998
K-Neighbors : n_neighbors = 35
Error: 0.08999999999999997


---
---
---
## Exercise 15
## Involved dataset: None
15. This problem involves writing functions.


In [50]:
# (a) Write a function, Power(), that prints out the result of raising 2 to the 3rd power.
# In other words, your function should compute 2^^3 and print out the results.
# Hint: Recall that x^a raises x to the power a. Use the print() function to output the result.
def Power():
    print(2 ** 3)

Power()


# (b) Create a new function, Power2(), that allows you to pass any
# two numbers, x and a, and prints out the value of x^a. You can
# do this by beginning your function with the line
#     > Power2 <- function (x, a) {
# You should be able to call your function by entering, for instance,
# > Power2(3, 8)
# on the command line. This should output the value of 3^8, namely, 6,561.
def Power2(x, a):
    print(x ** a)

Power2(3, 8)


# (c) Using the Power2() function that you just wrote, compute 10^3, 8^17, and 131^3.
Power2(10, 3)
Power2(8, 17)
Power2(131, 3)

# (d) Now create a new function, Power3(), that actually returns the result x^a as an R object, rather than simply printing it to the
# screen. That is, if you store the value x^a in an object calledresult within your function, then you can simply return() this return() result,
#  using the following line:
#     return (result)
# The line above should be the last line in your function, before the } symbol.
def Power3(x, b):
    return x ** b

result = Power3(2, 5)
print(result)

# (e) Now using the Power3() function, create a plot of f(x) = x^2.
# The x-axis should display a range of integers from 1 to 10, and
# the y-axis should display x2. Label the axes appropriately, and
# use an appropriate title for the figure. Consider displaying either
# the x-axis, the y-axis, or both on the log-scale. You can do this
# by using log = "x", log = "y", or log = "xy" as arguments to
# the plot() function.
list_x = list(range(1, 11))
list_y = [Power3(x, 2) for x in list_x]

import plotly.express as px
import pandas as pd

df = pd.DataFrame(dict(
    x = list_x,
    y = list_y
))
fig = px.line(df, x="x", y="y", title="Unsorted Input") 
fig.show()


# (f) Create a function, PlotPower(), that allows you to create a plot
# of x against x^a for a fixed a and for a range of values of x. For
# instance, if you call
#     > PlotPower(1:10, 3)
# then a plot should be created with an x-axis taking on values
# 1, 2,..., 10, and a y-axis taking on values 1^3, 2^3,..., 10^3.
def PlotPower(list_x:list, a:int):
    df = pd.DataFrame(dict(
        x = list_x,
        y = [Power3(x, a) for x in list_x]
    ))
    fig2 = px.line(df, x="x", y="y", title="Unsorted Input") 
    fig2.show()

PlotPower(list(range(100)), 2)

8
6561
1000
2251799813685248
2248091
32


---
---
---
## Exercise 16
## Involved dataset: Boston
16. Using the Boston data set, fit classification models in order to predict
whether a given census tract has a crime rate above or below the median. Explore logistic regression, LDA, naive Bayes, and KNN models
using various subsets of the predictors. Describe your findings.
Hint: You will have to create the response variable yourself, using the
variables that are contained in the Boston data set.

In [58]:
boston = pd.read_csv('drive/MyDrive/itsl_datasets/Boston.csv')
del boston['Unnamed: 0']
boston.describe()
# boston.head()

,crim,zn,indus,chas,nox,rm,age,dis,rad,tax,ptratio,lstat,medv
count,506.000000,506.000000,506.000000,506.000000,506.000000,506.000000,506.000000,506.000000,506.000000,506.000000,506.000000,506.000000,506.000000
mean,3.613524,11.363636,11.136779,0.069170,0.554695,6.284634,68.574901,3.795043,9.549407,408.237154,18.455534,12.653063,22.532806
std,8.601545,23.322453,6.860353,0.253994,0.115878,0.702617,28.148861,2.105710,8.707259,168.537116,2.164946,7.141062,9.197104
min,0.006320,0.000000,0.460000,0.000000,0.385000,3.561000,2.900000,1.129600,1.000000,187.000000,12.600000,1.730000,5.000000
25%,0.082045,0.000000,5.190000,0.000000,0.449000,5.885500,45.025000,2.100175,4.000000,279.000000,17.400000,6.950000,17.025000
50%,0.256510,0.000000,9.690000,0.000000,0.538000,6.208500,77.500000,3.207450,5.000000,330.000000,19.050000,11.360000,21.200000
75%,3.677083,12.500000,18.100000,0.000000,0.624000,6.623500,94.075000,5.188425,24.000000,666.000000,20.200000,16.955000,25.000000
max,88.976200,100.000000,27.740000,1.000000,0.871000,8.780000,100.000000,12.126500,24.000000,711.000000,22.000000,37.970000,50.000000


In [59]:
# (a) Create a binary variable, mpg01, that contains a 1 if mpg contains a value above its median, and a 0 if mpg contains a value below
# its median. You can compute the median using the median() function. Note you may find it helpful to use the data.frame()
# function to create a single data set containing both mpg01 and the other Auto variables.
median_crime = np.median(boston['crim'])
boston['crim_0'] = (boston['crim'] > median_crime).astype(int)

# Let's find features that are statistically significant
import plotly.express as px
numeric_columns = ['crim_0', 'zn', 'indus', 'chas', 'nox', 'rm', 'age', 'dis',
                   'rad', 'tax', 'ptratio', 'lstat', 'medv']

fig = px.scatter_matrix(boston[numeric_columns])
fig.update_traces(diagonal_visible=False)
fig.show()



import plotly.express as px
import numpy as np
corr_matrix = np.corrcoef(boston[numeric_columns], rowvar=False)

# corr_matrix
fig = px.imshow(corr_matrix
                , text_auto=True
                , labels=dict(x="Columns", y="Columns")
                , x=numeric_columns
                , y=numeric_columns
                , aspect="auto"
                , color_continuous_scale="BuPu"
                )
fig.update_xaxes(side="top")
fig.show()


In [78]:
# Extracting statitically important features that are above umbral of correlation (> 0.5)
dict_data = {
    'val_corr': corr_matrix[0],
    'val_corr_abs': np.abs(corr_matrix[0])
    }
df_crime = pd.DataFrame(dict_data, index=numeric_columns)
umbral = 0.5
df_crime = df_crime[(df_crime['val_corr_abs'] > umbral) & (df_crime['val_corr_abs'] != 1.0)]

predictors = df_crime.sort_values(by=['val_corr_abs'], ascending=False).index.to_list()
response = ['crim_0']

In [93]:
# Creating a function that evaluates different models
# logistic regression, LDA, naive Bayes, and KNN models
from sklearn.linear_model import LogisticRegression
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis, QuadraticDiscriminantAnalysis
from sklearn.naive_bayes import GaussianNB
from sklearn.neighbors import KNeighborsClassifier
from sklearn.model_selection import train_test_split

def evaluate_model(model, model_name, X_train, y_train, X_test, y_test):
    model.fit(X=X_train, y=y_train.values.ravel())
    # Evaluate the model
    y_pred = model.predict(X=X_test)
    score = model.score(X=X_test, y=y_test)
    print(f"Score for {model_name} model is: \t {score}")
    return score

models = [LogisticRegression(),
          LinearDiscriminantAnalysis(),
          QuadraticDiscriminantAnalysis(),
          GaussianNB(),
          KNeighborsClassifier(n_neighbors=5)]

models_name = ['Logistic Regression',
               'Linear Discriminant Analysis',
               'Quadratic Discriminant Analysis',
               'Naive Bayes',
               'K-Neighbors (K=5)']

# Split the data
X_train, X_test, y_train, y_test = train_test_split(boston[predictors], boston[response],
                                                    test_size=0.25, random_state=10)

best_score = -1
best_model_name = ''
for model_name, model in zip(models_name, models):
    score = evaluate_model(model, model_name,
                           X_train=X_train, y_train=y_train,
                           X_test=X_test, y_test=y_test)
    
    if score > best_score:
        best_score = score
        best_model_name = model_name


print("\n\n")    
print("=" * 100)
print("=" * 100)
print(f"The best model is {best_model_name} with a score of {best_score}")
print("=" * 100)
print("=" * 100, end='\n\n')


Score for Logistic Regression model is: 	 0.7952755905511811
Score for Linear Discriminant Analysis model is: 	 0.8267716535433071
Score for Quadratic Discriminant Analysis model is: 	 0.8818897637795275
Score for Naive Bayes model is: 	 0.7952755905511811
Score for K-Neighbors (K=5) model is: 	 0.8740157480314961



The best model is Quadratic Discriminant Analysis with a score of 0.8818897637795275



/usr/local/lib/python3.7/dist-packages/sklearn/linear_model/_logistic.py:818: ConvergenceWarning:

lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression

